In [ ]:
import json
import os
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import boto3
import datetime


def find_genre(original_list):
    new_list = []
    for item in original_list:
        if " " in item:
            new_list.extend(item.split())
        else:
            new_list.append(item)

    counts = {}
    for word in new_list:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    most_common_word = max(counts, key=counts.get)
    return most_common_word


def lambda_handler(event, context):
     # Step 1: Get your Spotify API credentials
    SPOTIPY_CLIENT_ID = os.environ.get('SPOTIPY_CLIENT_ID')
    SPOTIPY_CLIENT_SECRET = os.environ.get('SPOTIPY_CLIENT_SECRET')
    playlist_id = os.environ.get('playlist_id')
    # Step 2: Authenticate with the Spotify API
    auth_manager = SpotifyClientCredentials(client_id = SPOTIPY_CLIENT_ID, client_secret = SPOTIPY_CLIENT_SECRET)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    
    #get current date
    now = datetime.datetime.now()
    year = '{:02d}'.format(now.year)
    month = '{:02d}'.format(now.month)
    day = '{:02d}'.format(now.day)
    day_month_year = '{}-{}-{}'.format(year, month, day)
    

    
    # Step 3: Get the top 10 tracks for each country
    countries = ['China', 'India', 'United States', 'Indonesia', 'Pakistan', 'Brazil', 'Nigeria', 
                 'Bangladesh', 'Russia', 'Mexico', 'Japan', 'Ethiopia', 'Philippines', 'Egypt', 
                 'Vietnam', 'United Kingdom', 'France', 'Gernman', 'Australia']
    
    country_codes = ['CN', 'IN', 'US', 'ID', 'PK', 'BR', 'NG', 'BD', 'RU', 'MX', 'JP', 'ET', 
                     'PH', 'EG', 'VN','UK', 'FR', 'DE', 'AU']
    country_df = pd.DataFrame(
        {'country': countries,
         'country_code': country_codes
        })
    
    tracks = {}
    country_dfs = []
    c = 0
    for country in country_codes:
        df = []
        audio_features = []
        try:
            #sometimes, it does not have a playlist 
            playlist_id = sp.search(q=f'top 50 {country}', type='playlist')['playlists']['items'][0]['id']
            playlist_tracks = sp.playlist_tracks(playlist_id, limit=50)['items']
        except IndexError as e:
            print(country, 'missing playlist')
            continue
        for r in playlist_tracks:
            song_id = r['track']['id']
            song_name = r['track']['name']
    
            song_url = r['track']['external_urls']['spotify']
    
            song_popularity = r['track']['popularity']
    
            album_id = r['track']['album']['id']
    
            album_name = r['track']['album']['name']
            album_url = r['track']['album']['external_urls']['spotify']
            artist_id = r['track']['album']['artists'][0]['id']
            track_preview = track_preview = 'https://open.spotify.com/embed/track/'+ song_id
            try:
                album_image= r['track']['album']['images'][0]['url']
            except:
                try:
                    album_image = 'Null'
                except:
                    print('both code failed')
            artist_name = r['track']['album']['artists'][0]['name']
            results = sp.search(q = artist_name, type = 'artist')['artists']['items'][0]
            
            artist_type = results['genres']
            artist_followers = results['followers']['total']
            artist_popularity = results['popularity']
            artist_url = r['track']['album']['artists'][0]['external_urls']['spotify']
    
            try:
                artist_image = results['images'][0]['url']
            except:
                try:
                    artist_image = 'Null'
                except:
                    print('both code failed')
            record_date = day_month_year
            if len(artist_type) != 0 :
                genre = find_genre(artist_type)
            else:
                genre = 'Null'
            attribute = {'track_id': song_id,'track_name':song_name,'track_url':song_url,
                        'track_popularity':song_popularity, 
                        'album_id':album_id , 'album_name': album_name , 'countries':countries[c],
                        'album_url':album_url, 'album_image': album_image, 'track_preview' : track_preview,
                        'artist_id':artist_id , 'artist_name': artist_name, 'artist_url':artist_url, 'artist_image' :artist_image,
                        'artist_genre':genre, 'artist_followers': artist_followers, 'artist_popularity':artist_popularity,'record_date':record_date,
                        'Country': country, 'playlist_id': playlist_id
                }
    
            df.append(attribute)
            feature = sp.audio_features(song_url)[0]
            audio_features.append(feature)
        c = c+1
        # remove some attributes    
        rem_list = ['id', 'track_href','uri','type','analysis_url']
        for row in audio_features:
            for key in rem_list:
                if key in row:
                    del row[key]
        final_df = []
        for i in range(len(audio_features)):
            res = {**df[i],**audio_features[i]}
            final_df.append(res)                
        country_dfs.append(pd.DataFrame(final_df))
    track_by_country = pd.concat(country_dfs, ignore_index=True)
    print(track_by_country.columns)
    
    json_string = track_by_country.to_json()
    filename = 'spotify_raw_by_country'+ day_month_year + '.json'
    client = boto3.client('s3')

    
    # Upload the file to S3
    client.put_object(Bucket='spotify-project-jr', Key=key, Body=json_string)
    client.put_object(
        Bucket = 'spotify-project-jr',
        Key  = 'raw_data/by_country/' + filename,
        Body = json.dumps(json_string)
        )    
        



In [10]:
import os
import uuid
import base64
import cv2
import json
import numpy as np
from flask import Flask, request, jsonify
from threading import Thread
from yolo34py import YOLO, BBoxType

app = Flask(__name__)

yolo = YOLO("yolo_tiny_configs/yolov3-tiny.cfg",
            "yolo_tiny_configs/yolov3-tiny.weights",
            "yolo_tiny_configs/coco.names")

def detect_objects(image, image_id):
    img = np.array(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    detections = yolo.detect(img)

    detected_objects = []
    for detection in detections:
        label = detection.get_label()
        accuracy = detection.get_confidence()
        rectangle = detection.get_bbox(BBoxType.XYWH)

        detected_objects.append({
            'label': label,
            'accuracy': accuracy,
            'rectangle': {
                'height': int(rectangle.h),
                'left': int(rectangle.x),
                'top': int(rectangle.y),
                'width': int(rectangle.w)
            }
        })

    return {
        'id': image_id,
        'objects': detected_objects
    }

@app.route('/api/v1/detect_objects', methods=['POST'])
def api_detect_objects():
    data = request.get_json()
    image_id = data['id']
    image = base64.b64decode(data['image'])

    np_image = np.frombuffer(image, dtype=np.uint8)
    decoded_image = cv2.imdecode(np_image, flags=cv2.IMREAD_COLOR)

    thread = Thread(target=detect_objects, args=(decoded_image, image_id))
    thread.start()

    response = {
        'status': 'success',
        'message': 'Object detection started.'
    }
    return jsonify(response)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, threaded=True)

ModuleNotFoundError: No module named 'yolo34py'

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'F:\\pycharm\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
864071239543628